In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pprint import pprint
import seaborn as sb
#import altair as alt
from multiprocessing import Pool
#alt.renderers.enable('notebook')
import folium
from folium import plugins
from folium.plugins import HeatMap
%matplotlib inline
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#import pygeohash as pgh
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('ggplot')
plot_options = dict(figsize=(12,6))

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("Fires.csv")

C:\Users\johno\AppData\Local\Temp\ipykernel_18364\4076830362.py:1: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Fires.csv")


Below columns need to either be explicitly define type after reading in or dropped from DF

In [4]:
df.columns[[8,10,11,12,13,14,15,16,17,18,35,37]] 

Index(['SOURCE_REPORTING_UNIT', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID',
       'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'COUNTY', 'FIPS_NAME'],
      dtype='object')

In [5]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')

In [6]:
#dropping columns that will have little to no effect on model performance or contain NaN values
drop_cols = ['OBJECTID', 'FOD_ID', 'FPA_ID','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME','STAT_CAUSE_CODE','FIPS_CODE', 'FIPS_NAME', 'Shape','OWNER_CODE','DISCOVERY_TIME','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE']

#drop_cols = ['OBJECTID', 'FOD_ID', 'FPA_ID','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME','DISCOVERY_DATE','STAT_CAUSE_CODE','FIPS_CODE', 'FIPS_NAME', 'Shape','OWNER_CODE','DISCOVERY_TIME','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE']
df = df.drop(columns=drop_cols)

In [7]:
df.head()

,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE
0,FS,2005,2005-02-02,33,Miscellaneous,0.10,A,40.036944,-121.005833,USFS,CA
1,FS,2004,2004-05-12,133,Lightning,0.25,A,38.933056,-120.404444,USFS,CA
2,FS,2004,2004-05-31,152,Debris Burning,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA
3,FS,2004,2004-06-28,180,Lightning,0.10,A,38.559167,-119.913333,USFS,CA
4,FS,2004,2004-06-28,180,Lightning,0.10,A,38.559167,-119.933056,USFS,CA


Looking at all fires based on lat and long

In [8]:
years = list(df.FIRE_YEAR.unique())
years.sort()
for year in years:
    yearDf= df[df["FIRE_YEAR"]==year]
    print("Creating map with ", yearDf.size," fires ...")
    fig = px.scatter_geo(yearDf,lat='LATITUDE',lon='LONGITUDE',
                      size="FIRE_SIZE",
                      color = "FIRE_SIZE_CLASS")
    title = f"US Map of Fires(Year ={year})"
    fig.update_layout(title = title,geo_scope='usa', title_x=0.5)

    #fig.show()
    fig.write_image(f"USmapByYear/{year}_USmapFires.png")
    

Creating map with  747725  fires ...
Creating map with  681879  fires ...
Creating map with  835505  fires ...
Creating map with  786192  fires ...
Creating map with  831314  fires ...
Creating map with  675950  fires ...
Creating map with  752070  fires ...
Creating map with  982993  fires ...
Creating map with  1060576  fires ...
Creating map with  952457  fires ...
Creating map with  832216  fires ...
Creating map with  750871  fires ...
Creating map with  762069  fires ...
Creating map with  974644  fires ...
Creating map with  1254044  fires ...
Creating map with  1051303  fires ...
Creating map with  939158  fires ...
Creating map with  861575  fires ...
Creating map with  878779  fires ...
Creating map with  996072  fires ...
Creating map with  800459  fires ...
Creating map with  712580  fires ...
Creating map with  745283  fires ...
Creating map with  819401  fires ...


In [9]:
print("Creating map with all fires - size: ", df.size)
fig = px.scatter_geo(df,lat='LATITUDE',lon='LONGITUDE',
                  size="FIRE_SIZE",
                  color = "FIRE_SIZE_CLASS")
title = f"US Map of Fires"
fig.update_layout(title = title,geo_scope='usa', title_x=0.5)

#fig.show()
fig.write_image(f"USmapByYear/All_USmapFires.png")

Creating map with all fires - size:  20685115


In [60]:
#https://www.fs.usda.gov/about-agency/contact-us/regional-offices
#https://www.latlong.net/
#[ top left, bottom left, bottom right top right]
region1_coordinates = [(48.998625,-117.032181),(45.424841, -116.575238), (45.548077, -114.563236),(45.458889, -114.340904),(45.702748,-113.970683),(44.355692,-112.842069),(44.501816, -112.716047),(44.446140,-112.386114),(44.536460, -112.353637) ,(44.562305,-111.868789),(44.541651,-111.463793),(44.752791,-111.382034),(44.476095, -111.052484),(45.000642, -111.054512),(44.997898,-104.039652),(45.945135, -104.045442),(45.935240, -96.563685),(49.000315,-97.238073)]
region2_coordinates = [(44.476095, -111.052484),(44.476095,-110.001004),(44.159549,-109.050031),(41.000701,-109.050031),(36.998911, -109.045210),(36.689620,-109.045210),(36.689620,-103.002242),(37.000070,-103.002242),(37.000070,-94.617957),(39.101825,-94.617957),(42.435240, -96.420416),(45.301939, -96.420416),(45.604551, -96.855765),(45.935240, -96.563685),(45.00064,111.0545),(44.9979,104.0397),(45.94519,104.0454)]

In [ ]:
#requirements:
#pip install bokeh -not used
#pip install plotly
#pip install -U kaleido
#pip install shapely

In [61]:
def createRegionDF(coords,regionName,regionDF):
    coords = [ (coord[0],coord[1],regionName) for coord in coords]
    if(len(regionDF)==0 ):
        regionDF = pd.DataFrame(data= coords, index=None, columns=["lat","lon","region"] )
    else:
        newRegionDF = pd.DataFrame(data= coords, index=None, columns=["lat","lon","region"] )
        regionDF = pd.concat([regionDF,newRegionDF],axis=0)
    return regionDF
regionDF = createRegionDF(region1_coordinates,1,pd.DataFrame())
regionDF = createRegionDF(region2_coordinates,2,regionDF)

print("Regions in Dataframe ",regionDF["region"].unique())

Regions in Dataframe  [1 2]


In [64]:
regionDF[regionDF["region"]==2]

,lat,lon,region
0,44.476095,-111.052484,2
1,44.476095,-110.001004,2
2,44.159549,-109.050031,2
3,41.000701,-109.050031,2
4,36.998911,-109.045210,2
5,36.689620,-109.045210,2
6,36.689620,-103.002242,2
7,37.000070,-103.002242,2
8,37.000070,-94.617957,2
9,39.101825,-94.617957,2


In [78]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
 
def checkIfCoordInRegion(regionName,regionDF,df):#reference: https://stackoverflow.com/questions/61042261/how-to-check-if-a-lat-lon-coordinate-is-within-a-polygon-using-python-taking
    polygon = Polygon([tuple(x) for x in regionDF[['lat', 'lon']].to_numpy()])
    df[f'Within_{regionName}'] = df.apply(lambda x: polygon.contains(Point(x['LATITUDE'], x['LONGITUDE'])), axis=1)
    return df


for i in range(1,2):
    df = checkIfCoordInRegion(f"Region{i}",regionDF[regionDF["region"]==i],df)


     LATITUDE   LONGITUDE
57  48.921944 -115.092500
58  48.998333 -115.165556
60  47.665556 -115.569167
64  48.899444 -115.106944
65  48.906944 -115.076667


In [84]:
region1Coord_test =df[df["STATE"]== "MT"][0:5][["LATITUDE","LONGITUDE","Within_Region1"]]
print(region1Coord_test)

     LATITUDE   LONGITUDE  Within_Region1
57  48.921944 -115.092500            True
58  48.998333 -115.165556            True
60  47.665556 -115.569167            True
64  48.899444 -115.106944            True
65  48.906944 -115.076667            True


In [62]:
import plotly.express as px
#import plotly.graph_objects as go
import pandas as pd

fig = px.scatter_geo(regionDF,lat='lat',lon='lon',color = "region")
fig.update_layout(title = 'US map by region borders',geo_scope='usa', title_x=0.5)

#figLine = px.line(region1DF, x="lat", y="lon")
#figLine.update_traces(line=dict(color = 'rgba(50,50,50,0.2)'))

#fig3 = go.Figure(data=fig.data + figLine.data)
#fig3.show()

fig.show()

In [ ]:
# separation of training and testing data 
label_names = sorted(df['STAT_CAUSE_DESCR'].unique())
df_dummies_X = df.drop(columns=['STAT_CAUSE_DESCR'])
df_dummies_X = pd.get_dummies(df_dummies_X)
X = df_dummies_X
Y = df['STAT_CAUSE_DESCR']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.4, random_state=1, stratify=Y)

In [ ]:
df_dummies_X = df.drop(columns=['STAT_CAUSE_DESCR'])
df_dummies_X = pd.get_dummies(df_dummies_X)
X = df_dummies_X
Y = df['STAT_CAUSE_DESCR']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.4, random_state=1, stratify=Y)


classifier = RandomForestClassifier(n_estimators=500, max_depth=50, min_samples_leaf=50, random_state=123)

classifier.fit(X_train, Y_train)
training_accuracy = accuracy_score(Y_train, classifier.predict(X_train))
pred = classifier.predict(X_test)
test_accuracy = accuracy_score(Y_test, pred)

print(test_accuracy)

In [ ]:
%matplotlib inline
import seaborn as sb
import matplotlib.pyplot as plt
import altair as alt
label_names = sorted(df['STAT_CAUSE_DESCR'].unique())

matrix = confusion_matrix(Y_test, pred)
plt.figure(figsize=(8,8))
sb.heatmap(matrix,square=True,annot=True, fmt='d', cbar=False, cmap='Blues',
          xticklabels=label_names,yticklabels=label_names)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig("randomforestconfusionmatrix.png", dpi=170)